In [65]:
from coffea.nanoevents import NanoAODSchema,NanoEventsFactory
from coffea import processor
from coffea.analysis_tools import PackedSelection
import hist
from snippets import *
import awkward as ak
import numpy as np

In [66]:
events = NanoEventsFactory.from_root(
    "/commonfs/store/data/Run2018B/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/250000/7B7A01B5-FE11-5944-B73B-1FF6CB58CB83.root",
    schemaclass=NanoAODSchema.v7,
    metadata={
        "dataset":"MET_Run2018"
    }
).events()

In [67]:
eventsMC = NanoEventsFactory.from_root(
    "/commonfs/store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
    schemaclass=NanoAODSchema.v7,
    metadata={
        "dataset":"TTToSemiLeptonic"
    }
).events()

In [68]:
cutflow={}
cutflow["No cuts"] = len(events)

In [209]:
def single_tight_electrons_jjk(events, cutflow):
    # Etagap = ( events.Electron.eta < 1.4442 ) | ( events.Electron.eta > 1.566 )
    # elePassDXY = ((abs(events.Electron.eta) < 1.479) & (abs(events.Electron.dxy) < 0.05)) | ((abs(events.Electron.eta) > 1.479) & (abs(events.Electron.dxy) < 0.1))
    # elePassDZ = ((abs(events.Electron.eta) < 1.479) & (abs(events.Electron.dz) < 0.1)) | ((abs(events.Electron.eta) > 1.479) & (abs(events.Electron.dz) < 0.2))
    Eta = abs( events.Electron.eta ) < 2.5
    Pt = events.Electron.pt > 30.0 
    Id = events.Electron.cutBased == 4 #meaning only tight electrons 

    # final_cut = Etagap & elePassDXY & elePassDZ & Eta & Id & Pt
    final_cut =  Eta & Id & Pt
    all_tight = events.Electron[final_cut]
    #print(ak.sum(ak.num(all_tight)))
    one_tight_cut = ak.num(all_tight , axis=1) == 1
    goodevents = events[one_tight_cut]
    ha = events.Electron[final_cut]
    singles = ha[ak.num(ha)==1] 
    cutflow["one_tight_electron"] = len(goodevents)
    return goodevents , singles, cutflow

In [210]:
ak.sum(ak.num(events.Electron))

1033303

In [211]:
ak.sum(ak.num(events.Muon))

892602

In [212]:
electron_skimmed_events, single_electrons, cutflow = single_tight_electrons_jjk(events,cutflow)
muon_skimmed_events, single_muons, cutflow = single_tight_muons(events,cutflow)

In [213]:
ak.sum(ak.num(single_electrons))

10304

In [214]:
ak.sum(ak.num(single_muons))

42574

In [215]:
853500/1033303

0.8259919887970905

In [216]:
4257400/892602

4.769650975462748

In [217]:
events.Muon.tightId.__doc__

'cut-based ID, tight WP'

In [218]:
events.Muon.fields

['dxy',
 'dxyErr',
 'dxybs',
 'dz',
 'dzErr',
 'eta',
 'ip3d',
 'jetPtRelv2',
 'jetRelIso',
 'mass',
 'miniPFRelIso_all',
 'miniPFRelIso_chg',
 'pfRelIso03_all',
 'pfRelIso03_chg',
 'pfRelIso04_all',
 'phi',
 'pt',
 'ptErr',
 'segmentComp',
 'sip3d',
 'softMva',
 'tkRelIso',
 'tunepRelPt',
 'mvaLowPt',
 'mvaTTH',
 'charge',
 'jetIdx',
 'nStations',
 'nTrackerLayers',
 'pdgId',
 'tightCharge',
 'fsrPhotonIdx',
 'highPtId',
 'highPurity',
 'inTimeMuon',
 'isGlobal',
 'isPFcand',
 'isStandalone',
 'isTracker',
 'jetNDauCharged',
 'looseId',
 'mediumId',
 'mediumPromptId',
 'miniIsoId',
 'multiIsoId',
 'mvaId',
 'mvaLowPtId',
 'pfIsoId',
 'puppiIsoId',
 'softId',
 'softMvaId',
 'tightId',
 'tkIsoId',
 'triggerIdLoose',
 'cleanmask',
 'fsrPhotonIdxG',
 'jetIdxG']

In [219]:
events.Electron.cutBased.__doc__

'cut-based ID Fall17 V2 (0:fail, 1:veto, 2:loose, 3:medium, 4:tight)'